## News :D



In [4]:
%load_ext autoreload
%autoreload 2

import requests
import re 
import itertools
from bs4 import BeautifulSoup # Imports bs4
import wget # Imports wget
import articles
import sitemaps

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

### Data formats

CNN has links by month

NYT has links by day

BI has links by month

TH has links by day

NYP has links by day

DM has links by day

Fox has links by page num

### Thoughts on how to organize the data 

we use a pandas dataframe organized like this

| Dates | CNN | NYT | BI | TH | NYP | DM | FOX |
| ----------- | ------------------------------------------ | ------- | ------- | ------- | ------- | ------- | ------- |
| 2014-01-01  | list of tuples with article link & title   | ... | ... | ... | ... | ... | ... |
| 2014-01-02  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-03  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-04  | ... | ... | ... | ... | ... | ... | ... |
| ...         | ... | ... | ... | ... | ... | ... | ... |
| 2024-12-31  | ... | ... | ... | ... | ... | ... | ... |

### Code to get data
CNN:
```html
<li>
    <span class="date">2024-01-31</span>
    <span class="sitemap-link">
        <a href="https://www.cnn.com/tech/live-news/meta-x-discord-tiktok-snap-chiefs-testimony-senate/index.html">
            Mark Zuckerberg apologizes to families over social media harms in contentious Senate hearing
        </a>
    </span>
</li>
```

NYT:
```html
<li>
    <a href="https://www.nytimes.com/2023/12/31/business/dealbook/pga-tour-saudi-deal-deadline.html">
        PGA Tour and Saudi-Backed LIV Extend Deadline to Finalize Deal
    </a>
</li>
```

BI:
```html
<p>
    <a href="https://www.businessinsider.com/feud-capote-vs-swans-ann-woodward-suicide-death-real-story-2024-1">
        The real story behind the suicide of American socialite Ann Woodward in &#39;Feud: Capote vs. The Swans&#39;
    </a>
    <br>2024-02-01T03:31:01.68Z
</p>
```

NYP:
```html
<h3 class="story__headline headline headline--archive">
    <a href="https://nypost.com/2024/01/01/sports/og-anunoby-trending-toward-the-perfect-piece-for-knicks/" rel="" target="_self">
		Knicks&#039; big trade acquisition already looks like the &#039;perfect fit&#039; 												
    </a>
</h3>
```

DM:
```html
<a href="/wires/ap/article-12917573/Christian-McCaffrey-miss-49ers-regular-season-finale-return-playoffs.html" title="">
    Christian McCaffrey will miss the 49ers regular-season finale but should return for playoffs
</a>
```

FOX:
```html
<loc>
    https://www.foxnews.com/opinion/why-the-department-of-education-is-going-to-rip
</loc>
<lastmod>
    2025-03-12T05:00:54-04:00
</lastmod>
```

In [12]:
[cnn_dict, nyt_dict, bi_dict, nyp_dict, dm_dict, fox_links] = sitemaps.sitemaps()

In [ ]:
[cnn_articles, nyt_articles, bi_articles, nyp_articles, dm_articles, fox_articles] = articles.articles([0,0,0,0,0,0])

[('Georgia Downs Ohio State in Semifinal on Last-Minute Touchdown', 'https://www.nytimes.com/2023/01/01/sports/ncaafootball/peach-bowl-georgia-ohio-state.html'), ('The First Afterlife of Pope Benedict XVI', 'https://www.nytimes.com/2023/01/01/opinion/pope-benedict-xvi-catholic-church.html'), ('Quotation of the Day: ‘Today Girl’ to TV Icon, Walters Traversed News and Celebrity', 'https://www.nytimes.com/2023/01/01/pageoneplus/quotation-of-the-day-today-girl-to-tv-icon-walters-traversed-news-and-celebrity.html'), ('No Corrections: Jan. 1, 2023', 'https://www.nytimes.com/2023/01/01/pageoneplus/no-corrections-jan-1-2023.html'), ('Spelling Bee Answers: Sunday, January 1, 2022', 'https://www.nytimes.com/2023/01/01/crosswords/spelling-bee-forum.html'), ('What Should You Do Differently in 2023? Try Taking Suggestions.', 'https://www.nytimes.com/2023/01/01/style/resolutions-suggestions-advice.html'), ('‘Suddenly, It Appeared as Though the Bus Was Going to Be Able to Move’', 'https://www.nytimes

In [8]:
# Pandas Testing area
import pandas as pd

test_frame = pd.DataFrame()